In [15]:
import rasterio
import os
import snappy
from snappy import ProductIO, WKTReader, GPF

string = "S1A_IW_GRDH_1SDV_20240323T094000_20240323T094025_053107_066EC8_11F5"

sar_image_path = f'./data/final_{string}.dim'
product = ProductIO.readProduct(sar_image_path)

sar_image_path_tif = f'./data/final_{string}.tif'

In [16]:
with rasterio.open(sar_image_path_tif) as src:
    left, bottom, right, top = src.bounds
    print(left, bottom, right, top)

-61.09481282142434 -5.100124151952495 -58.489967992062965 -3.0889758938657104


In [17]:
left, bottom, right, top = -60, -4, -59.5, -3.5

bounding_wkt = f"POLYGON (({left} {top}, {left} {bottom}, {right} {bottom}, {right} {top}, {left} {top}))"
geometry = WKTReader().read(bounding_wkt)

In [18]:
SubsetOp = snappy.jpy.get_type('org.esa.snap.core.gpf.common.SubsetOp')

HashMap = snappy.jpy.get_type('java.util.HashMap')
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geometry)
product_subset = snappy.GPF.createProduct('Subset', parameters, product)

In [19]:
width = product_subset.getSceneRasterWidth()
print("Width: {} px".format(width))
height = product_subset.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = product_subset.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 5567 px
Height: 5567 px
Band names: Sigma0_VV, Sigma0_VH


In [20]:
ProductIO.writeProduct(product_subset, f"data/final_subset_{string}", 'BEAM-DIMAP')
os.path.exists(f"data/final_subset_{string}.dim")

True